In [66]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup as bs
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

base_url = "https://www.fueleconomy.gov/feg/"
makers = ["Chevrolet", "Nissan", "Ford"]
# Task: Columns -> Energy & Petrol Consumption, Storage

In [65]:
def get_data(base, maker):
    base_url = "https://www.fueleconomy.gov/feg/"
    maker_url = f"{base}PowerSearch.do?action=noform&path=4&year1=2019&year2=2024&make={maker}&srchtyp=newMake&pageno=1&rowLimit=50"
    response = r.get(maker_url)
    soup = bs(response.content)
    link = [x.get("href") for x in soup.select("a[href]")[154:210] if "yourMpgVehicle" not in x.get("href")]
    mpge = []
    mpg = []
    batter = []
    gas_tank = []
    for l in link:
        response = r.get(base_url+l)
        soup = bs(response.content)
        type = soup.find("td", class_="fuel nobottomborder padding").getText(strip=True)
        if type == "Electricity":
            
            mpge.append(soup.find("td", class_="combinedMPG").getText().split(":")[1])
            mpg.append("")
        else:
            gas_tank.append(soup.find("td",class_="sbsEconData")).getText().split(" ")[0]
            mpge.append("")
            mpg.append(soup.find("td", class_="combinedMPG").getText().split(":")[1])
    print(mpge,mpg)

get_data(base_url, "Nissan")

Electricity
Electricity
Electricity
Electricity
Electricity
Electricity
Electricity
Electricity
Electricity
Electricity
Electricity
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Premium Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
Regular Gasoline
['111', '111', '109', '109', '103', '101', '98', '95', '92', '90', '87', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 